<a href="https://colab.research.google.com/github/srehaag/JUR403-F24/blob/main/JUR_403.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JUR 403 - Using Computational Methods to Advance Social Justice

This notebook aims to help students in the Fall 2024 version of JUR 403 at TMU understand how generative AI can be used as a legal research tool to advance social justice.

#### Setup

In [1]:
!pip install -q datasets
!pip install -q langchain
!pip install -q langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 703.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 18.5 MB

#### Authenticate with OpenAI

Instructions:

1.   Get OpenAI developper platform account: https://platform.openai.com/signup
2.   Create & copy API key: https://platform.openai.com/settings/profile?tab=api-keys
3.   If using Colab, click on the key icon on the left, select add new secret, name the secret OPENAI_API_KEY & paste the key into the value field

In [2]:
# Authenticate with OpenAI using API key (for Colab users)
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

#### Load Refugee Law Lab Reporter Dataset from Huggingface

For further details, see: https://refugeelab.ca/bulk-data

In [3]:
# Load Data
from datasets import load_dataset
df = load_dataset("refugee-law-lab/canadian-legal-data", "RLLR", split="train").to_pandas()
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/641 [00:00<?, ? examples/s]

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
0,2022 RLLR 140,TC0-01092,RLLR,2022,,en,5/13/2022,https://refugeelab.ca/rllr/2022rllr140,2024-08-06,2022 RLLR 140\n\nCitation: 2022 RLLR 140\nTrib...,"{""country"": ""Bahamas"", ""case_type"": ""PSG: Gend..."
1,2022 RLLR 138,TB9-35046,RLLR,2022,,en,8/9/2022,https://refugeelab.ca/rllr/2022rllr138,2024-08-06,2022 RLLR 138\n\nCitation: 2022 RLLR 138\nTrib...,"{""country"": ""Iran"", ""case_type"": ""Political Op..."
2,2022 RLLR 137,TB9-30696,RLLR,2022,,en,9/26/2022,https://refugeelab.ca/rllr/2022rllr137,2024-08-06,2022 RLLR 137\n\nCitation: 2022 RLLR 137\nTrib...,"{""country"": ""Iran"", ""case_type"": ""Religion"", ""..."
3,2022 RLLR 136,TB9-27796,RLLR,2022,,en,1/18/2022,https://refugeelab.ca/rllr/2022rllr136,2024-08-06,2022 RLLR 136\n\nCitation: 2022 RLLR 136\nTrib...,"{""country"": ""Russia"", ""case_type"": ""Political ..."
4,2022 RLLR 135,TB9-27542,RLLR,2022,,en,5/4/2022,https://refugeelab.ca/rllr/2022rllr135,2024-08-06,2022 RLLR 135\n\nCitation: 2022 RLLR 135\nTrib...,"{""country"": ""China"", ""case_type"": ""Political O..."


In [4]:
# See example of text of a case
unofficial_text = df['unofficial_text'][250]
print (unofficial_text)

2020 RLLR 178

Citation: 2020 RLLR 178
Tribunal: Refugee Protection Division
Date of Decision: January 21, 2020
Panel: Randa Mekhael
Counsel for the Claimant(s): Marc J Herman
Country: Egypt
RPD Number: TB8-32026
Associated RPD Number(s): TB8-32090, TB8-32091
ATIP Number: A-2020-00518
ATIP Pages: 000793-000795
DECISION
[1] MEMBER: These are the reasons, the oral judgement for XXXX XXXX XXXX XXXX and her two children, XXXX XXXX XXXX and XXXX XXXX XXXX who claim to be citizens of Egypt and are claiming refugee protection pursuant to Section 96 and 97(1) of the Immigration and Refugee Protection Act.
[2] In rendering my decision, I considered the Chairperson’s Guidelines on Women Refugee Claimants Fearing Gender-Related Persecution. And I note that XXXX was the designated representative for both minors.
[3] With respect to your allegations, they are documented in the Basis of Claim form, Item 2.1, 2.2 and 2.3. And I’m not going to repeat them in this oral judgement.
[4] Just bare-, very b

#### Set up OpenAI calls via API

In [5]:
# Function to send true/false questions about a case to OpenAI GPT4o
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=1)
system_prompt = "You are a helpful assistant for a Canadian refugee law professor. You are provided with a case and asked a question about the case. You only answer true or false."

def get_response(system_prompt, question, unofficial_text):
  human_prompt = f"""CASE: {unofficial_text}\nQUESTION: {question}\nANSWER: """
  messages = [("system", system_prompt),("human", human_prompt)]
  return llm.invoke(messages).content

### Use OpenAI GPT4o to answer true/false questions about legal texts

In [6]:
question = "Is this a case about a refugee claim that primarily involves alleged persecution on account of the claimaint being transgender?"
get_response(system_prompt, question, unofficial_text)

'False'

In [7]:
# get 20 random case and see if is trans
df = df.sample(n=20, random_state=888).reset_index(drop=True)
df["is_trans"] = df["unofficial_text"].apply(lambda x: get_response(system_prompt, question, x))
df.head()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other,is_trans
0,2020 RLLR 187,VB9-01049,RLLR,2020,,en,3/20/2020,https://refugeelab.ca/rllr/2020rllr187,2024-08-06,2020 RLLR 187\n\nCitation: 2020 RLLR 187\nTrib...,"{""country"": ""Egypt"", ""case_type"": ""No Nexus: C...",False
1,2021 RLLR 31,TB9-22083,RLLR,2021,,en,9/28/2021,https://refugeelab.ca/rllr/2021rllr31,2024-08-06,2021 RLLR 31\n\nCitation: 2021 RLLR 31\nTribun...,"{""country"": ""Colombia"", ""case_type"": ""PSG: SOG...",True
2,2021 RLLR 79,TC0-01196,RLLR,2021,,en,6/4/2021,https://refugeelab.ca/rllr/2021rllr79,2024-08-06,2021 RLLR 79\n\nCitation: 2021 RLLR 79\nTribun...,"{""country"": ""Pakistan"", ""case_type"": ""Religion...",False
3,2021 RLLR 96,TC1-02935,RLLR,2021,,en,9/9/2021,https://refugeelab.ca/rllr/2021rllr96,2024-08-06,2021 RLLR 96\n\nCitation: 2021 RLLR 96\nTribun...,"{""country"": ""Sri Lanka"", ""case_type"": ""Politic...",False
4,2022 RLLR 133,TB9-05405,RLLR,2022,,en,3/23/2022,https://refugeelab.ca/rllr/2022rllr133,2024-08-06,2022 RLLR 133\n\nCitation: 2022 RLLR 133\nTrib...,"{""country"": ""China"", ""case_type"": ""Religion"", ...",False


In [8]:
# Show how many cases are trans cases
df.is_trans.value_counts()

,count
is_trans,
False,18
True,2


In [9]:
# Print the cases where is trans is true
trans_cases = df[df["is_trans"] == "True"]["unofficial_text"]
for case in trans_cases:
    print(case)

2021 RLLR 31

Citation: 2021 RLLR 31
Tribunal: Refugee Protection Division
Date of Decision: September 28, 2021
Panel: David Young
Counsel for the Claimant(s): Michael F. Loebach
Country: Colombia
RPD Number: TB9-22083
Associated RPD Number(s):
ATIP Number: A-2022-00665
ATIP Pages: 000155-000157
DECISION
MEMBER:
[1] I’ve reviewed the documents in this claim and asked the claimant some questions. I’m prepared to give my decision now.
[2] The claimant, XXXX XXXX XXXX is making a claim against Colombia, seeking refugee protection pursuant to section 96 of the Immigration and Refugee Protection Act.
[3] I find you are a convention refugee for the following reasons.
[4] The allegations are set out in the claimant’s basis of claim form. They’re quite detailed and consistent throughout the narrative. They do not raise any credibility concerns that need to be addressed on what I considered to be the key issue here.
[5] The allegations, as I see it, are on two parts. The first part is regarding

In [10]:
question = "Was the applicant successful in obtaining refugee protection in the case? Answer only true or false."
get_response(system_prompt, question, unofficial_text)

'True'

In [11]:
df["is_granted"] = df["unofficial_text"].apply(lambda x: get_response(system_prompt, question, x))
df.is_granted.value_counts()

,count
is_granted,
True,20
